In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

#following https://www.tensorflow.org/text/tutorials/text_generation

2025-07-17 11:34:06.124653: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    origin='https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',
    cache_dir='.',          # put it in the current working directory
    cache_subdir='data'     # ./data/shakespeare.txt
)

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(len(text)) 

1115394


In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [ ]:
stoi = {char: i for i, char in enumerate(vocab)}
itos = {i: char for i, char in enumerate(vocab)}
string_from_num = np.vectorize(itos.__getitem__)
num_from_string = np.vectorize(stoi.__getitem__)

In [7]:
all_ids = np.fromiter((stoi[ch] for ch in text), dtype=np.int32)
print(all_ids[:20])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


In [8]:
text_back = "".join(itos[id] for id in all_ids)
print(text_back[:20])

First Citizen:
Befor


In [9]:
all_ids.shape


(1115394,)

In [10]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [11]:
ids_dataset.take(1)

<_TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [12]:
for ids in ids_dataset.take(13):
    print(itos[int(ids)])

F
i
r
s
t
 
C
i
t
i
z
e
n


2025-07-17 11:34:09.639452: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:
seq_length = 100

In [14]:
print(itos)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [15]:
print([itos[i] for i in range(len(itos))])

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(2):
    ids_np = seq.numpy()
    # vocab_arr = np.array([itos[i] for i in range(len(itos))], dtype='<U1') works if lookup dictionary is 0-n
    # char_array = vocab_arr[ids_np]
    vocab_arr = np.vectorize(itos.__getitem__)
    char_array = vocab_arr(ids_np)
    print(char_array)

['F' 'i' 'r' 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'B' 'e' 'f'
 'o' 'r' 'e' ' ' 'w' 'e' ' ' 'p' 'r' 'o' 'c' 'e' 'e' 'd' ' ' 'a' 'n' 'y'
 ' ' 'f' 'u' 'r' 't' 'h' 'e' 'r' ',' ' ' 'h' 'e' 'a' 'r' ' ' 'm' 'e' ' '
 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'A' 'l' 'l' ':' '\n' 'S' 'p' 'e' 'a'
 'k' ',' ' ' 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'F' 'i' 'r' 's' 't' ' ' 'C'
 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'Y' 'o' 'u' ' ']
['a' 'r' 'e' ' ' 'a' 'l' 'l' ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd' ' ' 'r'
 'a' 't' 'h' 'e' 'r' ' ' 't' 'o' ' ' 'd' 'i' 'e' ' ' 't' 'h' 'a' 'n' ' '
 't' 'o' ' ' 'f' 'a' 'm' 'i' 's' 'h' '?' '\n' '\n' 'A' 'l' 'l' ':' '\n'
 'R' 'e' 's' 'o' 'l' 'v' 'e' 'd' '.' ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd'
 '.' '\n' '\n' 'F' 'i' 'r' 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':'
 '\n' 'F' 'i' 'r' 's' 't' ',' ' ' 'y' 'o' 'u' ' ' 'k']


2025-07-17 11:34:09.678530: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [18]:
dataset = sequences.map(split_input_target)

In [21]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [19]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [20]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    #map vocabs to tensor
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    #main program
    self.lstm = tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    #return most likely next charexp
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.lstm.get_initial_state(BATCH_SIZE)
    x, state_h, state_c = self.lstm(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [22]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


2025-07-17 11:38:34.739204: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [29]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 100, 256)         │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ((64, 100, 1024), (64, │     5,246,976 │
│                                 │ 1024), (64, 1024))     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 100, 65)          │        66,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [36]:
sampled_indices

array([50, 58, 56, 43,  1,  5, 43, 49, 43, 52,  1,  0, 37, 44,  1, 57, 56,
       39, 45, 44, 43,  1, 58, 58, 56,  6,  1, 39, 47, 47, 44, 63,  1, 63,
       47, 47, 58,  1, 39, 53, 59,  1, 42, 47, 57, 58,  1, 61, 44, 58, 39,
       41, 58, 47,  0,  0, 21, 17, 15, 21, 26, 32, 21, 27, 10,  0, 37, 53,
       43, 52, 43,  1, 58, 47, 54, 57,  6,  1, 39, 56, 39, 52, 47, 53,  6,
        0, 21, 43,  1, 50,  6, 58, 53, 57, 58,  1, 58, 46, 53, 59])

In [38]:
print(''.join((string_from_num(input_example_batch[0]))))
print("new line")
print(''.join((string_from_num(sampled_indices))))

asure ta'en:
In brief, sir, study what you most affect.

LUCENTIO:
Gramercies, Tranio, well dost tho
new line
ltre 'eken 
Yf sragfe ttr, aiify yiit aou dist wftacti

IECINTIO:
Yoene tips, aranio,
Ie l,tost thou


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) # loss function

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

In [ ]:
tf.exp(example_batch_mean_loss).numpy() # shows that because similar to vocab size, it is not sure of itself and eveyrthing is evenly spread

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
checkpoint_path = "ckpt_10.weights.h5"
model.load_weights(checkpoint_path)


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy